In [2]:
import numpy as np


In [628]:
class Derivatives(object):
    @staticmethod
    def sum(tensors, grad):
        return [np.ones(t.shape) * grad for t in tensors]

    @staticmethod
    def multiplication(tensors, g):
        grad = []
        for i, t1 in enumerate(tensors):
            partial_grad = np.ones(t1.shape)
            for j, t2 in enumerate(tensors):
                if i == j:
                    continue
                partial_grad *= t2
            partial_grad *= g
            grad.append(partial_grad)
        return grad

    @staticmethod
    def division(tensors, grad):
        if len(tensors) != 2:
            raise Exception('unexpected amount of tensors for division (currently only two operands are supported)')

        [f, g] = tensors
        return [
            grad / g,
            -f * grad / (g * g)
        ]

    @staticmethod
    def negation(tensors, grad):
        return [-1 * grad * np.ones(tensors[0].shape)]


In [839]:
class Tensor(object):

    @staticmethod
    def as_expr(t):
        if t.ntype == 'var':
            return str(t.data)
        if t.ntype == 'sum':
            return f'({Tensor.as_expr(t.children[0])} + {Tensor.as_expr(t.children[1])})'
        if t.ntype == 'mul':
            return f'({Tensor.as_expr(t.children[0])} * {Tensor.as_expr(t.children[1])})'
        if t.ntype == 'neg':
            return f'-({Tensor.as_expr(t.children[0])})'
        if t.ntype == 'div':
            return f'({Tensor.as_expr(t.children[0])} / {Tensor.as_expr(t.children[1])})'
    _id_generator = 0

    def __init__(self,
                 data,
                 autograd=True,
                 children=None,
                 derivative=None,
                 dtype=np.float64,
                 ntype="var"):

        if type(data) is not np.ndarray:
            data = np.array(data, dtype)
        else:
            data = data.astype(dtype)

        self.data = data
        self.grad = None
        self.autograd = autograd
        self.children = children
        self.derivative = derivative
        self.ntype = ntype
        self.dtype = dtype
        self.id = None
        self._generate_id()

    def _generate_id(self):
        self.id = Tensor._id_generator
        Tensor._id_generator += 1

    def __str__(self):
        return f'type=\'{self.ntype}\' data={self.data}'

    def __add__(self, other: 'Tensor'):
        return Tensor(data=self.data + other.data,
                      autograd=True,
                      children=(self, other),
                      derivative=Derivatives.sum,
                      dtype=self.dtype,
                      ntype='sum')

    def __mul__(self, other: 'Tensor'):
        return Tensor(data=self.data * other.data,
                      autograd=True,
                      children=(self, other),
                      derivative=Derivatives.multiplication,
                      dtype=self.dtype,
                      ntype='mul')

    def __neg__(self):
        return Tensor(data=-self.data,
                      autograd=True,
                      children=(self,),
                      derivative=Derivatives.negation,
                      dtype=self.dtype,
                      ntype='neg')

    def __sub__(self, other: 'Tensor'):
        a = self
        b = -other
        return b + a

    def __truediv__(self, other: 'Tensor'):
        return Tensor(data=self.data / other.data,
                      autograd=True,
                      children=(self, other),
                      derivative=Derivatives.division,
                      dtype=self.dtype,
                      ntype='div')

    # def cosine(self):

    def topsort(self, blacklist=None):
        if blacklist is None:
            blacklist = {}
        if self.id not in blacklist:
            blacklist[self.id] = 0
            yield self
        if self.children is not None:
            for child in self.children:
                for n in child.topsort(blacklist):
                    yield n

    def accumulate(self, grad):
        if self.grad is None:
            self.grad = grad
        else:
            self.grad += grad

    def backward(self):
        if not self.autograd:
            return

        if self.grad is None:
           self.grad = np.ones(self.data.shape, dtype=self.dtype)

        for n in self.topsort():
            if n.children is None:
                continue
            gradient = n.derivative([x.data for x in n.children], n.grad)
            for child, grad in zip(n.children, gradient):
                child.accumulate(grad)

    def zero_grad(self):
        self.grad = None
        for child in self.children:
            child.zero_grad()

In [806]:
A = 2342343432423324233234
B = 742324432

In [835]:
from torch import tensor

a = tensor(A, dtype=float, requires_grad=True)
b = tensor(B, dtype=float, requires_grad=True)
d =  ( b* (a + (-b)) * a) * b

d.backward()

print(float(a.grad))
print(float(b.grad))
# print(c.grad)
# print(d.grad)

-3.832398255368644e+68
-3.790757525216277e+67


In [836]:
a = Tensor(A, dtype=np.float64)
b = Tensor(B)
d =  (b * (a + (-b)) * a) * b
d.backward()
print(a.grad)
print(b.grad)
# print(Tensor.as_expr(d))
# print(c.grad)
# print(d.grad)

# for n in d.topsort():
#     print(n)

-3.832398255368644e+68
-3.790757525216277e+67


In [616]:
1.1077136540091577e+22 / 5.56156006205242e+21

1.9917318911420165

In [858]:
import random
import torch

torch.set_warn_always(True)
np.seterr('print')
def test(ops, max_leaves, max_ops):
    my_leaves = []
    torch_leaves = []

    for i in range(random.randint(1, max_leaves)):
        val = random.randint(1, 4)
        my_leaves.append(Tensor(val, dtype=np.float64))
        torch_leaves.append(torch.tensor(val, dtype=torch.float64, requires_grad=True))

    my_op = my_leaves[0]
    torch_op = torch_leaves[0]

    for i in range(random.randint(1, max_ops)):


        leaf_index = random.randint(0, len(my_leaves) - 1)
        op = random.choice(ops)
        my_op = op(my_op, my_leaves[leaf_index])
        torch_op = op(torch_op, torch_leaves[leaf_index])
        if torch_op.detach().numpy() != my_op.data:
            print('some failure!')


    my_op.backward()
    torch_op.backward()

    for i in range(len(my_leaves)):
        ml, tl = my_leaves[i].grad, torch_leaves[i].grad
        if ml is None or tl is None:
            if ml != tl:
                return False
            continue
        tl = tl.numpy()

        epsilon = max(abs(ml), abs(tl)) / 100
        if abs(ml - tl) > epsilon:
            print('failure:', ml, tl, len(my_leaves), len(torch_leaves), epsilon)
            # print(Tensor.as_expr(my_op))
            print(my_leaves[i].ntype)
            return False
    return True

OPS = [
    lambda a, b: a * b,
    lambda a, b: a + b,
    lambda a, b: a - b,
    lambda a, b: -a,
    lambda a, b: a + (-b),
    # lambda a, b: a / b
]

TESTS = 1000
MAX_LEAVES = 100
MAX_OPS = 1000

for i in range(1000):
    if not test(OPS, MAX_LEAVES, MAX_OPS):
        print('stopped at', i + 1)
        break
else:
    print('PASSED')


failure: -8153747712.0 -8053063680.0 74 74 81537477.12
var
stopped at 287


In [890]:
MB = [
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5],
    [1,2,3,4,5]
]

MA = [
    [2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2],
    [2, 2, 2, 2, 2]
]

MC = [
    [3,3,4,3,3],
    [3,3,4,3,3],
    [3,3,4,3,3],
    [3,3,4,3,3],
    [3,3,4,3,3]
]

In [903]:
a = tensor(MA, dtype=torch.float64, requires_grad=True)
b = tensor(MB, dtype=torch.float64, requires_grad=True)
c = tensor(MC, dtype=torch.float64, requires_grad=True)

d = a * b
e = b * c
f = d * d * e

f.backward(tensor([
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
    [1,1,1,1,1],
]))
print(b.grad)

tensor([[ 36., 144., 432., 576., 900.],
        [ 36., 144., 432., 576., 900.],
        [ 36., 144., 432., 576., 900.],
        [ 36., 144., 432., 576., 900.],
        [ 36., 144., 432., 576., 900.]], dtype=torch.float64)


In [904]:
a = Tensor(MA)
b = Tensor(MB)
c = Tensor(MC)

d = a * b
e = b * c
f = d * d * e


f.backward()
print(b.grad)

[[ 36. 144. 432. 576. 900.]
 [ 36. 144. 432. 576. 900.]
 [ 36. 144. 432. 576. 900.]
 [ 36. 144. 432. 576. 900.]
 [ 36. 144. 432. 576. 900.]]
